In [15]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

import tracker
import stats
import visualizer
import utils

import torch

import importlib

import os

In [4]:
# Get Data
video_fn = "../VISEM_Simple_Dataset/11.mp4"
labels_fn = "../VISEM_Simple_Dataset/11_labels.csv"

frames = utils.loadVideo(video_fn)
labels = utils.loadDataFrame(labels_fn)


In [ ]:
# Get a random sperm in the data

sperm_id = np.random.choice(labels["sperm"].unique(), 1)[0]
sperm = labels[labels["sperm"] == sperm_id]
# Get the sperm trajectory
sperm = sperm[["frame", "x", "y"]].to_numpy()

# Get the video for the same frames
video_frames = frames[sperm[:, 0].astype(int)]
sperm = sperm[:, 1:3]

# Normalize the x,y values
sperm[:, 0] = sperm[:, 0] / video_frames[0].shape[1]
sperm[:, 1] = sperm[:, 1] / video_frames[0].shape[0]


In [ ]:
# Use the model to predict the sperm trajectory
from predictLocation import LitNetwork

checkpoint = torch.load('best_model.ckpt')

#print(checkpoint.keys())

model = LitNetwork(2,2)
model.load_state_dict(checkpoint['state_dict'])
model.eval()
model.to("cuda")

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])


LitNetwork(
  (model): LSTMNetwork(
    (lstm): LSTM(2, 128, batch_first=True)
    (projection): Linear(in_features=128, out_features=2, bias=True)
  )
  (loss_func): MSELoss()
)

In [41]:
# Predict the sperm trajectory given the first 8 frames
predictions = sperm[:8].copy().tolist()

for i in range(8, len(sperm)):
    # Get the input data
    input_data = sperm[i-8:i].reshape(1, 8, 2).astype(np.float32)
    input_data = torch.from_numpy(input_data).to("cuda")

    # Predict the sperm trajectory
    with torch.no_grad():
        prediction = model(input_data).cpu().numpy()

    # Append the prediction to the list
    predictions.append(prediction[0])

# Convert the predictions to a numpy array
predictions = np.array(predictions)

In [ ]:
# Visualize the sperm trajectories
import cv2

cv2.imshow('frame', img)



(1470, 2) (1462, 2)
